In [1]:
from schnapsen.game import SchnapsenGamePlayEngine, Bot, PlayerPerspective, Move, RegularMove
from schnapsen.bots import RandBot, RdeepBot, BullyBot, MiniMaxBot, AlphaBetaBot
import random
from statistics import mean
import math
from scipy.stats import ttest_ind

engine = SchnapsenGamePlayEngine()

class SVBot(Bot):
    TRUMP_PENALTY = 10  

    def __init__(self, rand, name=None):
        super().__init__(name)
        self.rng = rand

    def get_move(self, perspective, leader_move):
        # Convert only regular moves
        valid_moves = []
        for move in perspective.valid_moves():
            if move.is_regular_move():
                valid_moves.append(move.as_regular_move())

        trump_suit = perspective.get_trump_suit()

        # If we start the trick -> leader logic
        if leader_move is None:
            return self.choose_as_leader(perspective, valid_moves, trump_suit)
        # Otherwise -> follower logic
        else:
            return self.choose_as_follower(perspective, valid_moves, leader_move, trump_suit)

    def get_rank_points(self, perspective, card):
        scorer = perspective.get_engine().trick_scorer
        return scorer.rank_to_points(card.rank)

    def get_card_cost(self, perspective, card, trump_suit):
        """
        Cost = rank points + TRUMP_PENALTY if the card is trump.
        """
        cost = self.get_rank_points(perspective, card)
        if card.suit == trump_suit:
            cost += self.TRUMP_PENALTY
        return cost

    def lowest_cost_move(self, perspective, moves, trump_suit):
        lowest_move = None
        lowest_cost = None

        for move in moves:
            cost = self.get_card_cost(perspective, move.card, trump_suit)
            if lowest_cost is None or cost < lowest_cost:
                lowest_cost = cost
                lowest_move = move

        return lowest_move

    def beats(self, my_card, other_card, trump_suit):
        # Same suit: higher rank wins
        if my_card.suit == other_card.suit:
            return my_card.rank.value > other_card.rank.value

        # Trump beats non-trump
        if my_card.suit == trump_suit and other_card.suit != trump_suit:
            return True

        return False


    def choose_as_leader(self, perspective, moves, trump_suit):

        trumps = []
        non_trumps = []
        good_non_trumps = []  # ACE, TEN, KING

        for m in moves:
            if m.card.suit == trump_suit:
                trumps.append(m)
            else:
                non_trumps.append(m)
                if m.card.rank.name in ["ACE", "TEN", "KING"]:
                    good_non_trumps.append(m)

        # If we have good non-trumps and trumps, compare cheapest of each
        if good_non_trumps and trumps:
            best_non_trump = self.lowest_cost_move(perspective, good_non_trumps, trump_suit)
            cheapest_trump = self.lowest_cost_move(perspective, trumps, trump_suit)

            cost_not_t = self.get_card_cost(perspective, best_non_trump.card, trump_suit)
            cost_trump = self.get_card_cost(perspective, cheapest_trump.card, trump_suit)

            # Pick the lower-cost option (trump or non-trump)
            if cost_trump < cost_not_t:
                return cheapest_trump
            return best_non_trump

        # Otherwise choose best available option
        if good_non_trumps:
            return self.lowest_cost_move(perspective, good_non_trumps, trump_suit)
        if non_trumps:
            return self.lowest_cost_move(perspective, non_trumps, trump_suit)

        # All are trump or no choice, so pick cheapest overall
        return self.lowest_cost_move(perspective, moves, trump_suit)

    def choose_as_follower(self, perspective, moves, leader_move, trump_suit):

        leader_card = leader_move.cards[0]

        winning_moves = []
        for m in moves:
            if self.beats(m.card, leader_card, trump_suit):
                winning_moves.append(m)

        # pick the lowest-cost winning card (trump or non-trump)
        if winning_moves:
            return self.lowest_cost_move(perspective, winning_moves, trump_suit)

        # dump the lowest-cost legal card
        return self.lowest_cost_move(perspective, moves, trump_suit)


In [7]:
#TESTING IF IT WORKS

engine = SchnapsenGamePlayEngine()

rand_bot = RandBot(random.Random(), name="randbot")
rdeep_bot = RdeepBot(depth = 4, num_samples = 45, rand = random.Random(), name="r_deep")

winner, game_points, game_score = engine.play_game(rand_bot, rdeep_bot, random.Random())

print(f"The winners is {winner} with {game_score.direct_points} points")

In [8]:
#TESTING IF IT WORKS


engine = SchnapsenGamePlayEngine()

rand_bot = RandBot(random.Random(), name="randbot")
sv_bot = SVBot(random.Random(), name="sv_bot")

winner, game_points, game_score = engine.play_game(rand_bot, sv_bot, random.Random())

print(f"The winner is {winner} with {game_score.direct_points} points")


In [9]:
# Average points RandBot vs SVBot (Experiment 1)

from schnapsen.bots import RandBot

rand_bot = RandBot(random.Random(), name="randbot")
sv_bot = SVBot(random.Random(), name="sv_bot")

rand_points = []
sv_bot_points = []

# Use a loop to run 100 games
for i in range(5001):
    # Run a game at each iteration of the loop and store the points 
    winner, game_score, game_points = engine.play_game(rand_bot, sv_bot, random.Random())
    if str(winner) == "randbot":
        rand_points.append(game_points.direct_points)
        sv_bot_points.append(0)
    else:
        rand_points.append(0)
        sv_bot_points.append(game_points.direct_points)

# This premade line will print the results if you completed the above code correctly
print(f"Rand has scored an average {mean(rand_points)}, while sv_bot scored an average {mean(sv_bot_points)}")

In [10]:
# Welch's t-test RandBot vs SVBot


from scipy.stats import ttest_ind
# RandBot vs SVBot
t_rand, p_rand = ttest_ind(
    sv_bot_points,
    rand_points,
    equal_var=False  # Welch's t-test
)
print("SVBot vs RandBot:")
print("t-statistic =", t_rand)
print("p-value =", p_rand)

In [2]:
# Average points BullyBot vs RandBot (Experiment 1)


rand_bot = RandBot(random.Random(), name="randbot")
bully_bot = BullyBot(random.Random(), name="bullybot")

rand_points = []
bully_points = []

# Use a loop to run 100 games
for i in range(5001):
    # Run a game at each iteration of the loop and store the points 
    winner, game_score, game_points = engine.play_game(rand_bot, bully_bot, random.Random())
    if str(winner) == "randbot":
        rand_points.append(game_points.direct_points)
        bully_points.append(0)
    else:
        rand_points.append(0)
        bully_points.append(game_points.direct_points)

# This premade line will print the results if you completed the above code correctly
print(f"Rand has scored an average {mean(rand_points)}, while bullybot scored an average {mean(bully_points)}")

In [3]:
# Welch's t-test RandBot vs BullyBot

from scipy.stats import ttest_ind
# RandBot vs BullyBot
t_rand, p_rand = ttest_ind(
    bully_points,
    rand_points,
    equal_var=False  # Welch's t-test
)
print("BullyBot vs RandBot:")
print("t-statistic =", t_rand)
print("p-value =", p_rand)

In [4]:
# Average points RdeepBot vs SVBot (Experiment 2)

rdeep_bot = RdeepBot(depth = 4, num_samples = 45, rand = random.Random(), name="r_deep")
sv_bot = SVBot(random.Random(), name="sv_bot")

rdeep_points = []
sv_bot_points = []

# Use a loop to run 100 games
for i in range(5001):
    # Run a game at each iteration of the loop and store the points 
    winner, game_score, game_points = engine.play_game(rdeep_bot, sv_bot, random.Random())
    if str(winner) == "r_deep":
        rdeep_points.append(game_points.direct_points)
        sv_bot_points.append(0)
    else:
        rdeep_points.append(0)
        sv_bot_points.append(game_points.direct_points)

# This premade line will print the results if you completed the above code correctly
print(f"RDeep has scored an average {mean(rdeep_points)}, while sv_bot scored an average {mean(sv_bot_points)}")

In [5]:
# Welch's t-test Rdeep vs SVBot


from scipy.stats import ttest_ind
# RdeepBot vs SVBot
t_rand, p_rand = ttest_ind(
    sv_bot_points,
    rdeep_points,
    equal_var=False  # Welch's t-test
)
print("SVBot vs RdeepBot:")
print("t-statistic =", t_rand)
print("p-value =", p_rand)

In [ ]:
# Experiment 3: SV vs Rdeep (Swapped Starts)

from schnapsen.game import SchnapsenGamePlayEngine
from schnapsen.bots import RdeepBot
from scipy.stats import ttest_ind
from statistics import mean, stdev
import random

# ---------- SETUP ----------
N = 5000
rng = random.Random(42)
engine = SchnapsenGamePlayEngine()

sv_bot = SVBot(rng, name="sv_bot")
rdeep_bot = RdeepBot(depth=4, num_samples=20, rand=rng, name="r_deep")

scores_start = []   # SV starts first
scores_second = []  # SV starts second
wins_start = 0
wins_second = 0

# ---------- CONDITION A: SV STARTS ----------
for _ in range(N):
    winner, game_score, game_points = engine.play_game(sv_bot, rdeep_bot, rng)

    # SV's direct points this game (same pattern you used before)
    if winner is sv_bot:
        sv_points = game_points.direct_points
        wins_start += 1
    else:
        sv_points = 0

    scores_start.append(sv_points)

# ---------- CONDITION B: SV SECOND ----------
for _ in range(N):
    winner, game_score, game_points = engine.play_game(rdeep_bot, sv_bot, rng)

    if winner is sv_bot:
        sv_points = game_points.direct_points
        wins_second += 1
    else:
        sv_points = 0

    scores_second.append(sv_points)

# ---------- RESULTS ----------
print("\n=== Experiment 2: SV vs Rdeep (Swapped Starts) ===")

print("\nSV STARTS FIRST:")
print(f"Mean score: {mean(scores_start):.2f}")
print(f"Std dev:    {stdev(scores_start):.2f}")
print(f"Win rate:   {wins_start / N:.3f}")

print("\nSV STARTS SECOND:")
print(f"Mean score: {mean(scores_second):.2f}")
print(f"Std dev:    {stdev(scores_second):.2f}")
print(f"Win rate:   {wins_second / N:.3f}")

# ---------- T-TEST ----------
t, p = ttest_ind(scores_start, scores_second, equal_var=False)
print("\nWelch t-test (start first vs second):")
print(f"t = {t:.3f}")
print(f"p = {p:.5f}")
